In [4]:
import pandas as pd
import numpy as np
import sweetviz
import joblib, pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as skmet

In [42]:
'''
Objective:
To automate the classification of glass types based on other features using the KNN algorithm, thereby streamlining the process and reducing manual effort in the glass manufacturing plant.
Constraints:
1.
Ensure the KNN algorithm is implemented effectively to achieve accurate classification results.
2.
Consider the availability and quality of data for training the KNN model.
3.
Address any potential computational constraints associated with implementing the KNN algorithm efficiently.
4.
Ensure the model's classification capabilities meet the accuracy requirements specified by the manufacturing plant.
5.
Consider scalability and maintenance aspects of the automated classification system for long-term viability.
'''
glass = pd.read_csv('glass.csv')
glass

RI     Na    Mg    Al     Si     K    Ca    Ba   Fe  Type
0    1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.00  0.0     1
1    1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.00  0.0     1
2    1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.00  0.0     1
3    1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.00  0.0     1
4    1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.00  0.0     1
..       ...    ...   ...   ...    ...   ...   ...   ...  ...   ...
209  1.51623  14.14  0.00  2.88  72.61  0.08  9.18  1.06  0.0     7
210  1.51685  14.92  0.00  1.99  73.06  0.00  8.40  1.59  0.0     7
211  1.52065  14.36  0.00  2.02  73.42  0.00  8.44  1.64  0.0     7
212  1.51651  14.38  0.00  1.94  73.61  0.00  8.48  1.57  0.0     7
213  1.51711  14.23  0.00  2.08  73.36  0.00  8.62  1.67  0.0     7

[214 rows x 10 columns]

In [6]:
glass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [7]:
glass.describe()

RI          Na          Mg          Al          Si           K  \
count  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000   
mean     1.518365   13.407850    2.684533    1.444907   72.650935    0.497056   
std      0.003037    0.816604    1.442408    0.499270    0.774546    0.652192   
min      1.511150   10.730000    0.000000    0.290000   69.810000    0.000000   
25%      1.516522   12.907500    2.115000    1.190000   72.280000    0.122500   
50%      1.517680   13.300000    3.480000    1.360000   72.790000    0.555000   
75%      1.519157   13.825000    3.600000    1.630000   73.087500    0.610000   
max      1.533930   17.380000    4.490000    3.500000   75.410000    6.210000   

               Ca          Ba          Fe        Type  
count  214.000000  214.000000  214.000000  214.000000  
mean     8.956963    0.175047    0.057009    2.780374  
std      1.423153    0.497219    0.097439    2.103739  
min      5.430000    0.000000    0.000000    1.000000  
25%      8.240000    0.000000    0.000000    1.000000  
50%      8.600000    0.000000    0.000000    2.000000  
75%      9.172500    0.000000    0.100000    3.000000  
max     16.190000    3.150000    0.510000    7.000000

In [8]:
glass.isna().sum()

RI      0
Na      0
Mg      0
Al      0
Si      0
K       0
Ca      0
Ba      0
Fe      0
Type    0
dtype: int64

In [10]:
x = glass.drop(['Type'], axis = 1)
y = glass['Type']

In [11]:
#imputation
num_pipeline = Pipeline(steps =[('impute', SimpleImputer(strategy = 'mean'))])
num_preprocessor = ColumnTransformer(transformers=[('numeric', num_pipeline, x.columns)])
impute_data = num_preprocessor.fit(x)
impute_df =pd.DataFrame(impute_data.transform(x), columns = x.columns)

In [19]:
#drop low variance features
impute_df=impute_df.drop(['Ba'], axis =1)
from feature_engine.outliers import Winsorizer
winsor = Winsorizer(capping_method ='iqr', tail = 'both', variables = list(impute_df.columns))
outlier_pipeline = Pipeline(steps =[('wins', winsor)])
outlier_data = outlier_pipeline.fit(impute_df)
outlier_df = pd.DataFrame(outlier_data.transform(impute_df), columns = impute_df.columns)

In [20]:
scale_pipeline = Pipeline(steps = [('scale', MinMaxScaler())])
scale_preprocessor = ColumnTransformer(transformers=[('scale', scale_pipeline, impute_df.columns)])
scale_data = scale_preprocessor.fit(outlier_df)
scale_df = pd.DataFrame(scale_data.transform(outlier_df), columns = outlier_df.columns)

In [21]:
joblib.dump(impute_data, 'knn_assignment_impute')
joblib.dump(outlier_data, 'knn_assignment_outlier')
joblib.dump(scale_data, 'knn_assignment_scale')

['knn_assignment_scale']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(scale_df, y, test_size = 0.2, random_state =0)


#model building

In [24]:
knn = KNeighborsClassifier(n_neighbors = 21)
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=21)

In [25]:
pred_train = knn.predict(x_train)
pd.crosstab(y_train, pred_train, rownames =['Actual'], colnames=['prediction'])

prediction   1   2   7
Actual                
1           54   7   0
2           23  33   1
3           11   1   0
5            2   5   4
6            3   0   4
7            3   2  18

In [27]:
skmet.accuracy_score(y_train, pred_train)

0.6140350877192983

In [28]:
pred = knn.predict(x_test)
skmet.accuracy_score(y_test, pred)

0.5813953488372093

In [29]:
pd.crosstab(y_test, pred, rownames=['Actual'], colnames=['predictions'])

predictions  1   2  7
Actual               
1            8   1  0
2            7  12  0
3            4   1  0
5            0   1  1
6            1   0  1
7            1   0  5

In [30]:
skmet.confusion_matrix(y_test, pred)

array([[ 8,  1,  0,  0,  0,  0],
       [ 7, 12,  0,  0,  0,  0],
       [ 4,  1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  1],
       [ 1,  0,  0,  0,  0,  1],
       [ 1,  0,  0,  0,  0,  5]], dtype=int64)

In [31]:
# Hyperparameter optimization
from sklearn.model_selection import GridSearchCV
k = list(range(3, 50,2))
param_grid = dict(n_neighbors = k)

In [33]:
grid = GridSearchCV(knn, param_grid, cv = 5, scoring = 'accuracy', return_train_score = False, verbose = 1)

In [34]:
grid.fit(x_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_neighbors=21),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23,
                                         25, 27, 29, 31, 33, 35, 37, 39, 41, 43,
                                         45, 47, 49]},
             scoring='accuracy', verbose=1)

In [35]:
grid.best_params_

{'n_neighbors': 3}

In [36]:
#accuracy
grid.best_score_*100

67.17647058823529

In [37]:
pred_grid = grid.predict(x_test)

In [38]:
pd.crosstab(y_test, pred_grid, rownames = ['actual'], colnames=['prediction'])

prediction  1   2  3  5  6  7
actual                       
1           6   3  0  0  0  0
2           8  11  0  0  0  0
3           4   0  1  0  0  0
5           0   0  0  2  0  0
6           0   0  0  0  1  1
7           1   0  0  0  0  5

In [39]:
grid.best_estimator_

KNeighborsClassifier(n_neighbors=3)

In [40]:
prediction = pd.DataFrame(grid.predict(x_test), columns =['glass_prediction'])

In [41]:
prediction

glass_prediction
0                  7
1                  1
2                  1
3                  6
4                  5
5                  2
6                  1
7                  2
8                  2
9                  1
10                 2
11                 2
12                 1
13                 2
14                 2
15                 7
16                 1
17                 2
18                 3
19                 1
20                 5
21                 1
22                 7
23                 1
24                 2
25                 1
26                 7
27                 1
28                 2
29                 2
30                 1
31                 1
32                 2
33                 1
34                 1
35                 1
36                 1
37                 1
38                 1
39                 7
40                 2
41                 7
42                 2